# Requesting Comments

1. Insert your API Key from https://console.cloud.google.com/apis/api/youtube.googleapis.com to 'DEVELOPER_KEY' in your '.env' as a string.
2. Insert your video id to 'videoId' as string. Example: https://www.youtube.com/watch?v=SIm2W9TtzR0, videoId='SIm2W9TtzR0'

**Import DEVELOPER_KEY, IMPORTANT!**<br>
**RUN THIS FIRST BEFORE ANYTHING ELSE**

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
DEVELOPER_KEY = os.getenv('DEVELOPER_KEY')

**Example**

In [ ]:
import googleapiclient.discovery
import googleapiclient.errors

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part='snippet',
    videoId='0FtcHjI5lmw',
    maxResults=100
)
response = request.execute()
response

# for item in response['items']:
#     print(item['snippet']['topLevelComment']['snippet']['textDisplay'])

# Requesting Comments to DataFrame

### **Request for specific channel**

**Imports & Youtube API Init**

In [2]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

**Request Video Ids using Search** <br>
**IMPORTANT! VERY EXPENSIVE (100 quota per search)**

In [ ]:
CHANNEL_ID = ''

request_video_ids = youtube.search().list(
    part='snippet',
    channelId=CHANNEL_ID,
    type='video',
)
response_video_ids = request_video_ids.execute()

video_ids = []
    
while True:
    for item in response_video_ids['items']:
        video_ids.append(item['id']['videoId'])
    try:
        next_page_token_search = response_video_ids['nextPageToken']
    except KeyError:
        break
    request_video_ids = youtube.search().list(
        part='snippet',
        channelId=CHANNEL_ID,
        type='video',
        pageToken=next_page_token_search
    )
    response_video_ids = request_video_ids.execute()

**Request Video Ids using Playlist Items** <br>
**Cheaper! (1 quota per call)**

Info: The playlist ids for a channel is the same except for character at index 1, which is replaced with a 'U'. Ex: CHANNEL_ID = 'UCq22aK0t0mrOEq676Be4ezw' => PLAYLIST_ID = 'UUq22aK0t0mrOEq676Be4ezw'

In [22]:
CHANNEL_ID = 'UCq22aK0t0mrOEq676Be4ezw'

def replace_str_index(text,index=0,replacement=''):
    return f'{text[:index]}{replacement}{text[index+1:]}'

PLAYLIST_ID = replace_str_index(CHANNEL_ID, 1, 'U')

request_video_ids = youtube.playlistItems().list(
    part='snippet',
    playlistId=PLAYLIST_ID,
)
response_video_ids = request_video_ids.execute()

video_ids = []

while True:
    for item in response_video_ids['items']:
        video_ids.append(item['snippet']['resourceId']['videoId'])
    try:
        next_page_token_playlist = response_video_ids['nextPageToken']
    except KeyError:
        break
    request_video_ids = request_video_ids = youtube.playlistItems().list(
        part='snippet',
        playlistId=PLAYLIST_ID,
        pageToken=next_page_token_playlist
    )
    response_video_ids = request_video_ids.execute()

**Request Comments for every Video Id**

In [26]:
comments = []
errors = []

comments = []

for video_id in video_ids:
    try:
        request_comments = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=1000
        )
        response_comments = request_comments.execute()

        while True:
            for item in response_comments['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                comments.append([
                    comment['authorDisplayName'],
                    comment['publishedAt'],
                    comment['likeCount'],
                    comment['textOriginal']
                ])
            next_page_token_comments = response_comments['nextPageToken']
            request_comments = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=1000,
                pageToken=next_page_token_comments
            )
            response_comments = request_comments.execute()

    except Exception as e:
        if 'commentsDisabled' in str(e):
            errors.append(f"Comments are disabled for video {video_id}. Skipping...")
            continue
        else:
            errors.append(f"An error occurred for video {video_id}: {e}")
            continue

df = pd.DataFrame(comments, columns=['author', 'published_at', 'like_count', 'text'])

In [ ]:
# df
# df.head()
# df.sort_values('published_at', ascending=False)
df.sort_values('like_count', ascending=False)

**Convert to CSV**

In [38]:
df.to_csv('comments_from_blumechannel.csv', index=False)